In [2]:
import torch, torchvision, pathlib
import numpy as np
import pyvips
from PIL import Image
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler
from sklearn.decomposition import PCA

In [3]:
# Using de_1m_2013_extended-debuffered as reference
cvprDataPath = pathlib.Path('/home/users/jeafilho/dataset/cvpr_chesapeake_landcover/')
trainImgPaths = (cvprDataPath / 'de_1m_2013_extended-debuffered-train_tiles').glob('*leaf-on.tif')
valImgPaths = (cvprDataPath / 'de_1m_2013_extended-debuffered-val_tiles').glob('*leaf-on.tif')
testImgPaths = (cvprDataPath / 'de_1m_2013_extended-debuffered-test_tiles').glob('*leaf-on.tif')

In [4]:
# Creating train data
imgs = []
masks = []
stop = False
for path in trainImgPaths:
    img = pyvips.Image.new_from_file(str(path))
    imgs.append(img.numpy())
    mask = pyvips.Image.new_from_file(str(path).removesuffix('landsat-leaf-on.tif') + 'lc.tif')
    masks.append(mask.numpy())
    if stop: break
    stop = True


In [24]:
# 1 Scaler per band, since the range of every band is different
scalers = []
nBands = imgs[0].shape[2]
nBands = 2
for i in range(nBands):
    dummyArray = np.array([[]])
    for img in imgs:
        dummyArray = np.concatenate([dummyArray, img[..., i].reshape(-1,1)], axis=None)
    scalers.append(MinMaxScaler().fit(dummyArray.reshape(-1,1)))
#     print(np.concatenate(imgs, axis=2).shape)
    # scalers.append(MinMaxScaler().fit(np.concatenate(imgs, axis=2)[i::nBands].reshape((-1,1))))

In [26]:
scalers[1].data_max_

array([3604.])

In [30]:
for i in range(nBands):
    for img in imgs:
        shp = img.shape
        np.vectorize(scalers[i].transform(img[:,:,i].reshape(-1,1)))

In [84]:
for i in range(img.shape[2]):
    scaler = MinMaxScaler()
    img[:,:,i] = scaler.fit_transform(np.squeeze(img[:,:,i]))


In [85]:
xlen = 256
ylen = 256
dest = []
img = img.astype(np.float32)
mask = mask.astype(np.float32)
for i in range(img.shape[0]//xlen):
    for j in range(img.shape[1]//ylen):
        _a = np.concatenate([img[xlen*i:xlen*(i+1),ylen*j:ylen*(j+1)],np.expand_dims(mask[xlen*i:xlen*(i+1),ylen*j:ylen*(j+1)], 2)],2)
        dest.append(_a)
final = np.stack(dest)
final.shape

(432, 256, 256, 10)

In [86]:
np.save('/home/users/jeafilho/dataset/original5', final)